In [18]:
import random
(q0, q1, q2) = (7, 11, 13) # RNS basis of size 3
l = [q0, q1, q2]
q=1
p1 = 17 # p1 is the message modulus
p2 = 23
#print(p1*p2)
for x in l:
    q = q*x

# q is ciphertext modulus. 391 = p1.p2 < q = 1001
#print(q)
Z = Integers()
Zq = Integers(q)
Zq0 = Integers(q0)
Zq1 = Integers(q1)
Zq2 = Integers(q2)
Zp1 = Integers(p1)
# Define the polynomial rings
Rm.<x> = PolynomialRing(Zp1, 'x')
R.<x> = PolynomialRing(Zq, 'x')
R0.<x> = PolynomialRing(Zq0, 'x')
R1.<x> = PolynomialRing(Zq1, 'x')
R2.<x> = PolynomialRing(Zq2, 'x')

# Define the quotient ring Z_q[x]/(x^n + 1)
n = 8
Qmod = R.quotient(x^n + 1, 'x')
Q0 = R0.quotient(x^n + 1, 'x')
Q1 = R1.quotient(x^n + 1, 'x')
Q2 = R2.quotient(x^n + 1, 'x')
Rmod = Rm.quotient(x^n + 1, 'x')

q0_t = q//q0
q1_t = q//q1
q2_t = q//q2

q0_t_inverse = Zq0(1)/Zq0(q0_t)
q1_t_inverse = Zq1(1)/Zq1(q1_t)
q2_t_inverse = Zq2(1)/Zq2(q2_t)


crt_moduli = [p1, p2] # CRT modulus 

e_0 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x^2) + crt([0,1],crt_moduli)*Qmod(x^5) + crt([0,1],crt_moduli)*Qmod(x^7) ) # This is encode(0, 1 + x^2 + x^5 + x^7)
e_1 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x) + crt([0,1],crt_moduli)*Qmod(x^4) + crt([0,1],crt_moduli)*Qmod(x^6) )
e_2 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x^3) + crt([0,1],crt_moduli)*Qmod(x^6) + crt([0,1],crt_moduli)*Qmod(x^7) )

random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_0 = Qmod(random_polynomial)
random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_1 = Qmod(random_polynomial)
random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_2 = Qmod(random_polynomial)

s = Qmod(1 + x + x^3 + x^5 + x^6 + x^7 ) # s is the secret key

#print(s)
# Generation of public key:

random_polynomial = sum([Zq(random.randint(0, 5))*Qmod(x^i) for i in range(n)])
mask = Qmod(random_polynomial)
e = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x) + crt([0,1],crt_moduli)*Qmod(x^2) + crt([0,1],crt_moduli)*Qmod(x^3) + crt([0,1],crt_moduli)*Qmod(x^5) + crt([0,1],crt_moduli)*Qmod(x^7) )

public_0 = Qmod(mask*s + e)
public_1 = Qmod(-(mask))

beta_0 = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*s*s - alpha_0*s + e_0)
beta_1 = Qmod(Zq(q1_t)*Zq(q1_t_inverse)*s*s - alpha_1*s + e_1)
beta_2 = Qmod(Zq(q2_t)*Zq(q2_t_inverse)*s*s - alpha_2*s + e_2)

#Reduction of beta_0, beta_1, beta_2, alpha_0, alpha_1, alpha_2 in RNS basis [q0, q1, q2]:

def reduce_coefficients(f,p):
    f_parent = f.lift()
    coeffs = f_parent.coefficients(sparse=False)
    reduced_coeffs = [Z(c) % p for c in coeffs]
    f_reduced = sum(reduced_coeffs[i] * x^i for i in range(len(reduced_coeffs)))
    return f_reduced

beta_00 = Q0(reduce_coefficients(beta_0, q0))
beta_01 = Q1(reduce_coefficients(beta_0, q1))
beta_02 = Q2(reduce_coefficients(beta_0, q2))

beta_10 = Q0(reduce_coefficients(beta_1, q0))
beta_11 = Q1(reduce_coefficients(beta_1, q1))
beta_12 = Q2(reduce_coefficients(beta_1, q2))

beta_20 = Q0(reduce_coefficients(beta_2, q0))
beta_21 = Q1(reduce_coefficients(beta_2, q1))
beta_22 = Q2(reduce_coefficients(beta_2, q2))

alpha_00 = Q0(reduce_coefficients(alpha_0, q0))
alpha_01 = Q1(reduce_coefficients(alpha_0, q1))
alpha_02 = Q2(reduce_coefficients(alpha_0, q2))

alpha_10 = Q0(reduce_coefficients(alpha_1, q0))
alpha_11 = Q1(reduce_coefficients(alpha_1, q1))
alpha_12 = Q2(reduce_coefficients(alpha_1, q2))

alpha_20 = Q0(reduce_coefficients(alpha_2, q0))
alpha_21 = Q1(reduce_coefficients(alpha_2, q1))
alpha_22 = Q2(reduce_coefficients(alpha_2, q2))


In [19]:
def generate_encryption(m):
    l = [0,1]
    random_polynomial = sum([Zq(random.randint(0, 2))*Qmod(x^i) for i in range(n)])
    u = Qmod(random_polynomial)
    random_crt_polynomial = sum([Zq(crt([m[i],random.choice(l)],crt_moduli))*Qmod(x^i) for i in range(n)])
    encode1 = Qmod(random_crt_polynomial)
    random_crt_polynomial = sum([Zq(crt([0,random.choice(l)],crt_moduli))*Qmod(x^i) for i in range(n)])
    encode2 = Qmod(random_crt_polynomial)
    
    c_0 = Qmod(public_0*u + encode1)
    c_1 = Qmod(public_1*u + encode2)
    return (c_0, c_1)

def decryption(c_0, c_1):
    
    T = Qmod(c_0 + c_1*s)
    T_parent = T.lift()
    coeffs = T_parent.coefficients(sparse=False)
    reduced_coeffs = [Z(c) % p1 for c in coeffs]
    T_reduced = sum(reduced_coeffs[i] * Rmod(x^i) for i in range(len(reduced_coeffs)))
    return T_reduced



In [20]:
def multiplication(a_0, b_0, c_0, d_0): # (a_0, b_0) and (c_0, d_0) are 1st and 2nd ciphertexts respectively to multiply by relinearization
    h_0 = a_0*b_0
    h_1 = a_0*d_0 + b_0*c_0
    h_2 = b_0*d_0
    
    h_20 = Qmod(reduce_coefficients(h_2, q0))
    h_21 = Qmod(reduce_coefficients(h_2, q1))
    h_22 = Qmod(reduce_coefficients(h_2, q2))
    
    g_0 = beta_0*h_20 + beta_1*h_21 + beta_2*h_22
    g_1 = alpha_0*h_20 + alpha_1*h_21 + alpha_2*h_22
    
    return (h_0 + g_0, h_1 + g_1)

#Relinearization in RNS basis [q0, q1, q2]:

def multiplication_in_RNSbasis(a_0, b_0, c_0, d_0):
    h_0 = a_0*b_0
    h_1 = a_0*d_0 + b_0*c_0
    h_2 = b_0*d_0
    
    h_20 = Qmod(reduce_coefficients(h_2, q0))
    h_21 = Qmod(reduce_coefficients(h_2, q1))
    h_22 = Qmod(reduce_coefficients(h_2, q2))
    
    h_200 = Q0(reduce_coefficients(h_20, q0))
    h_201 = Q1(reduce_coefficients(h_20, q1))
    h_202 = Q2(reduce_coefficients(h_20, q2))
    
    h_210 = Q0(reduce_coefficients(h_21, q0))
    h_211 = Q1(reduce_coefficients(h_21, q1))
    h_212 = Q2(reduce_coefficients(h_21, q2))
    
    h_220 = Q0(reduce_coefficients(h_22, q0))
    h_221 = Q1(reduce_coefficients(h_22, q1))
    h_222 = Q2(reduce_coefficients(h_22, q2))
    
    L0 = Qmod(reduce_coefficients(beta_00*h_200 + beta_10*h_210 + beta_20*h_220, q))
    L1 = Qmod(reduce_coefficients(beta_01*h_201 + beta_11*h_211 + beta_21*h_221, q))
    L2 = Qmod(reduce_coefficients(beta_02*h_202 + beta_12*h_212 + beta_22*h_222, q))
    
    K0 = Qmod(reduce_coefficients(alpha_00*h_200 + alpha_10*h_210 + alpha_20*h_220, q))
    K1 = Qmod(reduce_coefficients(alpha_01*h_201 + alpha_11*h_211 + alpha_21*h_221, q))
    K2 = Qmod(reduce_coefficients(alpha_02*h_202 + alpha_12*h_212 + alpha_22*h_222, q))
    
    g_0 = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*L0 + Zq(q1_t)*Zq(q1_t_inverse)*L1 + Zq(q2_t)*Zq(q2_t_inverse)*L2)
    g_1 = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*K0 + Zq(q1_t)*Zq(q1_t_inverse)*K1 + Zq(q2_t)*Zq(q2_t_inverse)*K2)
    
    return (h_0 + g_0, h_1 + g_1)



In [27]:

m_1 = [0, 11, 0, 0, 0, 0, 7, 11]
m_2 = [3, 4, 8, 12, 13, 0, 10, 3]
m_1poly = Rmod(sum(m_1[i]*Rmod(x^i) for i in range(n)))
m_2poly = Rmod(sum(m_2[i]*Rmod(x^i) for i in range(n)))
print(m_1poly,'\n', m_2poly, '\n m_1*m_2=',m_1poly*m_2poly)

(a_0, b_0) = generate_encryption(m_1)
(c_0, d_0) = generate_encryption(m_2)
print("Encryption of m_1:", a_0, "and", b_0)
print("Encryption of m_2:", c_0, "and", d_0)
print("Decryption for m_1:",decryption(a_0, b_0))
print("Decryption for m_2:",decryption(c_0, d_0))
(v_0, v_1) = multiplication_in_RNSbasis(a_0, b_0, c_0, d_0)
print("Multiplication of encrypted m_1 and encrypted m_2:",multiplication_in_RNSbasis(a_0, b_0, c_0, d_0))
print("Decryption of multiplication:",decryption(v_0, v_1))

11*x^7 + 7*x^6 + 11*x 
 3*x^7 + 10*x^6 + 13*x^4 + 12*x^3 + 8*x^2 + 4*x + 3 
 m_1*m_2= x^7 + 5*x^6 + 12*x^5 + 11*x^4 + 13*x^3 + 8*x^2 + 14*x + 3
Encryption of m_1: 582*x^7 + 952*x^6 + 525*x^5 + 554*x^4 + 916*x^3 + 970*x^2 + 317*x + 746 and 974*x^7 + 990*x^6 + 315*x^5 + 5*x^4 + 17*x^3 + 349*x^2 + 347*x + 28
Encryption of m_2: 50*x^7 + 798*x^6 + 902*x^5 + 373*x^4 + 308*x^3 + 790*x^2 + 544*x + 531 and 304*x^7 + 988*x^6 + 319*x^5 + 2*x^4 + 332*x^3 + 342*x^2 + 341*x + 18
Decryption for m_1: 11*x^6 + 4*x^5 + 15*x^2 + 9*x + 9
Decryption for m_2: 9*x^7 + 14*x^6 + 2*x^5 + 13*x^4 + 12*x^3 + 6*x^2 + 2*x + 12
Multiplication of encrypted m_1 and encrypted m_2: (546*x^7 + 321*x^6 + 605*x^5 + 735*x^4 + 468*x^3 + 661*x^2 + 686*x + 180, 346*x^7 + 773*x^6 + 223*x^5 + 550*x^4 + 430*x^3 + 506*x^2 + 881*x + 292)
Decryption of multiplication: 7*x^7 + 11*x^6 + 5*x^5 + 15*x^4 + 14*x^3 + 6*x^2 + 6*x + 3
